<a href="https://colab.research.google.com/github/Desmondonam/NLP_1/blob/main/Speech_1_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We'll use the Common Voice dataset from Mozilla for speech data. This dataset can be accessed via the datasets library, which provides easy access to various datasets

In [ ]:
# Setting up the environment
!pip install librosa tensorflow seaborn matplotlib


In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requir

In [ ]:
# PRactice
from datasets import load_dataset, Audio

In [ ]:
dataset = load_dataset("PolyAI/minds14", "en-US", split="train")

The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset.column_names

['path',
 'audio',
 'transcription',
 'english_transcription',
 'intent_class',
 'lang_id']

In [ ]:
from datasets import load_dataset
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load the Common Voice dataset
# dataset = load_dataset("mozilla-foundation/common_voice_6_1", "en", split='train[:1%]')
# dataset = load_dataset("PolyAI/minds14", "en-US", split="train")
dataset = load_dataset("PolyAI/minds14", "en-US", split="train")



# Extract audio samples and corresponding text
audio_samples = []
transcriptions = []

for sample in dataset:
    audio_samples.append(sample['path'])
    transcriptions.append(sample['english_transcription'])

# Display sample data
print(f"Sample Audio Path: {audio_samples[0]}")
print(f"Sample Transcription: {transcriptions[0]}")

# Load and preprocess the audio data
def load_audio(audio_path):
    y, sr = librosa.load(audio_path, sr=16000)
    return y, sr

# Extract features (MFCCs) from audio samples
def extract_features(audio_path):
    y, sr = load_audio(audio_path)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return mfccs.mean(axis=1)

# Apply feature extraction
features = np.array([extract_features(path) for path in audio_samples])

# Encode transcriptions
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(transcriptions)
transcriptions_encoded = tokenizer.texts_to_sequences(transcriptions)
transcriptions_padded = tf.keras.preprocessing.sequence.pad_sequences(transcriptions_encoded, padding='post')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, transcriptions_padded, test_size=0.2, random_state=42)


Sample Audio Path: /root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US~JOINT_ACCOUNT/602ba55abb1e6d0fbce92065.wav
Sample Transcription: I would like to set up a joint account with my partner


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# Reshape data for RNN
X_train_rnn = np.expand_dims(X_train, axis=-1)
X_test_rnn = np.expand_dims(X_test, axis=-1)

# Define the RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(transcriptions_padded.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
# model.fit(X_train_rnn, y_train, epochs=10, batch_size=32, validation_data=(X_test_rnn, y_test))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 67), output.shape=(None, 67)

# 3. Building Machine Learning Models
3.1. Speech-to-Text: Recurrent Neural Network (RNN)
We’ll build a simple RNN model for speech-to-text using the extracted features.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# Reshape data for RNN
X_train_rnn = np.expand_dims(X_train, axis=-1)
X_test_rnn = np.expand_dims(X_test, axis=-1)

# Define the RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(transcriptions_padded.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_rnn, y_train, epochs=10, batch_size=32, validation_data=(X_test_rnn, y_test))


NameError: name 'X_train' is not defined